# **Task:** count publications by language (GIGO example)

The field `languageCode` is **shortcut** for the content of the structured field [`language`](https://graph.openaire.eu/docs/data-model/entities/research-product#language)

In [ ]:
%%bigquery

SELECT languageCode, COUNT(*) AS n_papers
FROM oag_v9_0_1.publications
GROUP BY languageCode
ORDER BY n_papers DESC
LIMIT 25

Query is running:   0%|          |

Downloading:   0%|          |

,languageCode,n_papers
0,und,162151831
1,eng,10207588
2,fra/fre,2923939
3,deu/ger,2560258
4,jpn,2396616
5,rus,2372113
6,ita,1669119
7,tur,1309483
8,Español,757917
9,spa,725757


# **Task:** find publications with a keyword in the title, e.g., "covid"

In [ ]:
%%bigquery

SELECT * # check the dry run before running the query!
FROM oag_v9_0_1.publications
WHERE CONTAINS_SUBSTR(mainTitle, 'covid')
LIMIT 5

In [ ]:
%%bigquery

SELECT mainTitle, authors
FROM oag_v9_0_1.publications
WHERE CONTAINS_SUBSTR(mainTitle, 'covid')
LIMIT 5

Let's manipalate the author structure

In [ ]:
%%bigquery

SELECT id, mainTitle, ARRAY_AGG(JSON_VALUE(author.fullName))
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(authors)) AS author
WHERE CONTAINS_SUBSTR(mainTitle, 'covid')
GROUP BY id, mainTitle
LIMIT 10

# **Task:** count publications per year


In [ ]:
%%bigquery

SELECT publicationDate AS year, COUNT(*) AS n_pubs
FROM oag_v9_0_1.publications
GROUP BY year
ORDER BY year DESC

Ouch! There are very strange-looking years in the query results... Let's check why!

Grab one year with just one publication and check it, e.g., `9999-01-01`

In [ ]:
%%bigquery

SELECT id
FROM oag_v9_0_1.publications
WHERE publicationDate = '9999-01-01'
LIMIT 10

The suggested query returns id `od______3686::02f2bfd3e9d1d278a9ba84f7f4658b43`

This can be fed to the following URL https://explore.openaire.eu/search/publication?articleId=***the_id*** to see the record on OpenAIRE portal.
e.g., https://explore.openaire.eu/search/publication?articleId=od______3686::02f2bfd3e9d1d278a9ba84f7f4658b43

If you click to view the source record (https://iris.uniroma1.it/handle/11573/414149), you will see that the metadata field is off from the source.

So, we need to extract years from dates **AND** we need to filter by year interval.

*Hint:* There is a handy function to extract the year from the relevant field `publicationDate`. Check the following.

In [ ]:
%%bigquery

SELECT EXTRACT(YEAR FROM DATE('2024-01-20')) as YEAR

Query is running:   0%|          |

Downloading:   0%|          |

,YEAR
0,2024


Let's plug now that function into the previous query.

In [ ]:
%%bigquery

SELECT EXTRACT(YEAR FROM DATE(publicationDate)) AS year,
        COUNT(*) AS n_pubs
FROM oag_v9_0_1.publications
GROUP BY year
ORDER BY year DESC

Let's try to filter away year distortions. Repeat the query above by providing a feasible year range.

In [ ]:
%%bigquery

SELECT EXTRACT(YEAR FROM DATE(publicationDate)) AS year, count(*) AS n_pubs
FROM oag_v9_0_1.publications
GROUP BY year
HAVING year BETWEEN 2014 AND 2024
# HAVING year >= 2014 AND year <= 2024
ORDER BY year DESC

## **Subtask:** count publications with a given keyword in the title, aggregated by year

In [ ]:
%%bigquery

SELECT EXTRACT(YEAR FROM DATE(publicationDate)) AS year, count(*) AS n_pubs
FROM oag_v9_0_1.publications
WHERE CONTAINS_SUBSTR(mainTitle, 'covid')
GROUP BY year
HAVING year BETWEEN 2014 AND 2024
# HAVING year >= 2014 AND year <= 2024
ORDER BY year DESC

# **Task:** count publications by country (of the organisations managing data sources)



In [ ]:
%%bigquery

SELECT JSON_VALUE(pub_country.code) AS code, COUNT(DISTINCT id) AS n_pubs
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(countries)) AS pub_country
GROUP BY code
ORDER BY n_pubs DESC
LIMIT 10

## **Subtask:** count publications by country in a specific year

In [ ]:
%%bigquery

SELECT JSON_VALUE(pub_country.code) AS code, COUNT(DISTINCT id) AS n_pubs
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(countries)) AS pub_country
WHERE EXTRACT(YEAR FROM DATE(publicationDate)) = 2023
GROUP BY code
ORDER BY n_pubs DESC

In [ ]:
# hasAuthorInstitution

## Subtask: add keyword filtering to WHERE clause

In [ ]:
%%bigquery

SELECT JSON_VALUE(pub_country.code) AS code, COUNT(DISTINCT id) AS n_pubs
FROM
  oag_v9_0_1.publications,
    UNNEST(JSON_QUERY_ARRAY(countries)) AS pub_country
WHERE EXTRACT(YEAR FROM DATE(publicationDate)) = 2023 AND
      CONTAINS_SUBSTR(mainTitle, 'crispr')
GROUP BY code
ORDER BY n_pubs DESC

# **Task:** find active projects

In [ ]:
%%bigquery

SELECT id, title,
    EXTRACT(YEAR FROM DATE(projects.startDate)) AS start_year,
    EXTRACT(YEAR FROM DATE(projects.endDate)) AS end_year
FROM oag_v9_0_1.projects
WHERE EXTRACT(YEAR FROM DATE(projects.endDate)) > 2024

## Subtask: count active projects by jurisdiction

To find active projects within a specific jurisdiction works like this

In [ ]:
%%bigquery

SELECT id, title, JSON_VALUE(p_fund.jurisdiction) AS jurisdiction,
  EXTRACT(YEAR FROM DATE(projects.startDate)) AS start_year,
  EXTRACT(YEAR FROM DATE(projects.endDate)) AS end_year
FROM oag_v9_0_1.projects,
  UNNEST(JSON_QUERY_ARRAY(fundings)) AS p_fund
WHERE EXTRACT(YEAR FROM DATE(projects.endDate)) > 2024 AND
  JSON_VALUE(p_fund.jurisdiction) = 'EU'
LIMIT 5

Then, we can aggregate using that field

In [ ]:
%%bigquery

SELECT JSON_VALUE(p_fund.jurisdiction) AS jurisdiction, COUNT(id) AS n_projects
FROM oag_v9_0_1.projects,
  UNNEST(JSON_QUERY_ARRAY(fundings)) AS p_fund
WHERE EXTRACT(YEAR FROM DATE(projects.endDate)) > 2024
GROUP BY jurisdiction
ORDER BY n_projects DESC

# **Task:** count projects by subject

In [ ]:
%%bigquery

SELECT project_subjects
FROM oag_v9_0_1.projects,
  UNNEST(JSON_QUERY_ARRAY(subjects)) AS project_subjects

In [ ]:
%%bigquery

SELECT JSON_VALUE(project_subjects) AS subject, COUNT(id) AS n_projects
FROM oag_v9_0_1.projects,
  UNNEST(JSON_QUERY_ARRAY(subjects)) AS project_subjects
GROUP BY subject
ORDER BY n_projects DESC

## **Subtask:** count active projects by subjects

In [ ]:
%%bigquery

SELECT JSON_VALUE(subject) as subject, COUNT(id) AS n_projects
FROM oag_v9_0_1.projects,
  UNNEST(JSON_QUERY_ARRAY(subjects)) AS subject
WHERE EXTRACT(YEAR FROM DATE(projects.endDate)) > 2024
GROUP BY subject
ORDER BY n_projects DESC


## **Subtask:** count project by subject, aggregate total funded amount

Hint: Funded amounts can have different currency.

In [ ]:
%%bigquery

SELECT JSON_VALUE(subject) AS subject,
        COUNT(id) AS n_projects,
        SUM(fundedAmount) AS funded_total
FROM
  oag_v9_0_1.projects,
    UNNEST(JSON_QUERY_ARRAY(subjects)) AS subject
WHERE currency = 'EUR'
GROUP BY subject
ORDER BY n_projects DESC, funded_total DESC

# **Task:** count different OA statuses by year

In [ ]:
%%bigquery

SELECT EXTRACT(YEAR FROM DATE(publicationDate)) as year,
      JSON_VALUE(bestAccessRight.label) AS OA_status,
      COUNT(*) AS n_papers
FROM oag_v9_0_1.publications
GROUP BY year, OA_status
HAVING year BETWEEN 2020 AND 2024
ORDER BY year DESC

# **Task:** explore instances

In [ ]:
%%bigquery

SELECT JSON_VALUE(instance.type) as instance_type, COUNT(*) AS count
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(instances)) AS instance
GROUP BY instance_type
ORDER BY count DESC

## **Subtask:** filter by instance types

The table `publications` has this many rows

In [ ]:
%%bigquery

SELECT COUNT(DISTINCT id)
FROM oag_v9_0_1.publications

However, we may want to restrict to specific instance types, for example

In [ ]:
%%bigquery

SELECT COUNT(DISTINCT id)
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(instances)) AS instance
WHERE JSON_VALUE(instance.type) IN ('Article',
                                           'Book',
                                           'Conference object',
                                           'Part of book or chapter of book',
                                           'Data Paper',
                                           'Software Paper')

In [ ]:
%%bigquery

SELECT *
FROM oag_v9_0_1.publications
WHERE id IN (SELECT DISTINCT id
              FROM oag_v9_0_1.publications,
                UNNEST(JSON_QUERY_ARRAY(instances)) AS instance
              WHERE JSON_VALUE(instance.type) IN ('Article',
                                                        'Book',
                                                        'Conference object',
                                                        'Part of book or chapter of book',
                                                        'Data Paper',
                                                        'Software Paper'))
LIMIT 10

## Subtask: focus on peer-reviewed material

Transform this query, focusing only on peer-reviewed material.

Hint: peer-review information can be found inside [instances](https://graph.openaire.eu/docs/data-model/entities/other#instance), i.e., the field `refereed`

In [ ]:
%%bigquery

SELECT JSON_VALUE(country.code) AS code, COUNT(DISTINCT id) AS n_pubs
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(countries)) AS country
GROUP BY code
ORDER BY n_pubs DESC
LIMIT 10

In [ ]:
%%bigquery

SELECT JSON_VALUE(country.code) AS code, COUNT(DISTINCT id) AS n_pubs
FROM oag_v9_0_1.publications,
  UNNEST(JSON_QUERY_ARRAY(countries)) AS country
WHERE id IN (SELECT DISTINCT id
              FROM oag_v9_0_1.publications,
                UNNEST(JSON_QUERY_ARRAY(instances)) AS instance
              WHERE JSON_VALUE(instance.refereed) = 'peerReviewed')
GROUP BY code
ORDER BY n_pubs DESC
LIMIT 10

# **Task:** aggregate publications by author

In [ ]:
%%bigquery

SELECT JSON_VALUE(author.fullName) AS fullname,
COUNT(DISTINCT(id)) AS n_pubs
FROM oag_v9_0_1.publications,
UNNEST(JSON_QUERY_ARRAY(authors)) author
GROUP BY fullname
LIMIT 50

In [ ]:
%%bigquery

SELECT JSON_VALUE(pub_author.pid.id.value) AS orcid,
    JSON_VALUE(pub_author.fullName) AS fullname,
    COUNT(DISTINCT(id)) AS n_pubs
FROM oag_v9_0_1.publications,
    UNNEST(JSON_QUERY_ARRAY(authors)) pub_author
WHERE JSON_VALUE(pub_author.pid.id.value) IS NOT NULL
GROUP BY orcid, fullname
ORDER BY n_pubs DESC